In [19]:
import logging
logger = logging.getLogger()

In [2]:
import pandas as pd

In [3]:
csv_path = ""

In [6]:
df = pd.read_csv(csv_path, index_col=0)

In [ ]:
df.head()

In [32]:
nexus_header = '''#NEXUS
[written by csv2nex - jam.muskopf@gmail.com]
'''

fmt_taxa_name = lambda label: label.replace(' ', '_')
taxa_labels = " ".join([fmt_taxa_name(taxa_label) for taxa_label in df.index.values])
nexus_taxa = f'''
BEGIN TAXA;
	TITLE Taxa;
	DIMENSIONS NTAX={len(df.index)};
	TAXLABELS
		{taxa_labels}
	;

END;
'''

In [33]:
set(df.values.flatten())

{'0', '1', '2', '3', '4', '5', '?'}

In [34]:
fmt_cstate_label = lambda label: label.replace("'", '"')
character_state_labels = ",\n".join([f"\t\t{i} '{fmt_cstate_label(name)}'" for i, name in enumerate(df.columns, start=1)])

missing = '?'
cstates = set(df.values.flatten())
cstates = [s for s in cstates if s != missing]

valid_states = list()
for state in cstates:
    try:
        int(state)
    except:
        logger.warning(f'Cannot cast character state {state} as integer; filling with "{missing}"')
        df[df == state] = missing
    else:
        valid_states.append(state)

valid_states.sort()
    

max_len_name = max([len(taxa_name) for taxa_name in df.index.values])
pad_taxa_name = lambda name: ' '*(max_len_name - len(name) + 1)
encode_row = lambda row: "".join(row.values.tolist())

matrix = '\n'.join([f'\t{fmt_taxa_name(taxa_name)}{pad_taxa_name(taxa_name)}{encode_row(row)}' for taxa_name, row in df.T.items()])

nexus_character = f'''
BEGIN CHARACTERS;
	TITLE  Character_Matrix;
	DIMENSIONS  NCHAR={len(df.columns)};
	FORMAT DATATYPE = STANDARD RESPECTCASE GAP = - MISSING = {missing} SYMBOLS = "  {" ".join(valid_states)}";
	CHARSTATELABELS
{character_state_labels} ;
	MATRIX
{matrix}

;
END;
'''


In [35]:
with open('result.nex', 'w') as f:
    f.write("\n".join([nexus_header, nexus_taxa, nexus_character]))